# GAN to Point Cloud

In [ ]:
#@title Install
%tensorflow_version 1.x
import tensorflow as tf

# Download the code
!git clone https://github.com/NVlabs/stylegan2-ada.git
%cd stylegan2-ada

import glob
import argparse
import numpy as np
import dnnlib
import dnnlib.tflib as tflib
import re
import sys
from io import BytesIO
import IPython.display
from math import ceil
import PIL
from PIL import Image, ImageDraw
import imageio
import os
import pickle
from google.colab import files
import struct
import cv2

In [3]:
#@title Additional functions for image generation
# Generates a list of images, based on a list of latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_in_w_space(dlatents, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    Gs_kwargs.truncation_psi = truncation_psi
    dlatent_avg = Gs.get_var('dlatent_avg') # [component]

    imgs = []
    for row, dlatent in log_progress(enumerate(dlatents), name = "Generating images"):
        #row_dlatents = (dlatent[np.newaxis] - dlatent_avg) * np.reshape(truncation_psi, [-1, 1, 1]) + dlatent_avg
        dl = (dlatent-dlatent_avg)*truncation_psi   + dlatent_avg
        row_images = Gs.components.synthesis.run(dlatent,  **Gs_kwargs)
        imgs.append(PIL.Image.fromarray(row_images[0], 'RGB'))
    return imgs       

def generate_images(zs, truncation_psi):
    Gs_kwargs = dnnlib.EasyDict()
    Gs_kwargs.output_transform = dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True)
    Gs_kwargs.randomize_noise = False
    if not isinstance(truncation_psi, list):
        truncation_psi = [truncation_psi] * len(zs)
        
    imgs = []
    for z_idx, z in log_progress(enumerate(zs), size = len(zs), name = "Generating images"):
        Gs_kwargs.truncation_psi = truncation_psi[z_idx]
        noise_rnd = np.random.RandomState(1) # fix noise
        tflib.set_vars({var: noise_rnd.randn(*var.shape.as_list()) for var in noise_vars}) # [height, width]
        images = Gs.run(z, None, **Gs_kwargs) # [minibatch, height, width, channel]
        imgs.append(PIL.Image.fromarray(images[0], 'RGB'))
    return imgs

def generate_zs_from_seeds(seeds):
    zs = []
    for seed_idx, seed in enumerate(seeds):
        rnd = np.random.RandomState(seed)
        z = rnd.randn(1, *Gs.input_shape[1:]) # [minibatch, component]
        zs.append(z)
    return zs

# Generates a list of images, based on a list of seed for latent vectors (Z), and a list (or a single constant) of truncation_psi's.
def generate_images_from_seeds(seeds, truncation_psi):
    return generate_images(generate_zs_from_seeds(seeds), truncation_psi)

def saveImgs(imgs, location):
  for idx, img in log_progress(enumerate(imgs), size = len(imgs), name="Saving images"):
    file = location+ str(idx) + ".png"
    img.save(file)

def imshow(a, format='png', jpeg_fallback=True):
  a = np.asarray(a, dtype=np.uint8)
  str_file = BytesIO()
  PIL.Image.fromarray(a).save(str_file, format)
  im_data = str_file.getvalue()
  try:
    disp = IPython.display.display(IPython.display.Image(im_data))
  except IOError:
    if jpeg_fallback and format != 'jpeg':
      print ('Warning: image was too large to display in format "{}"; '
             'trying jpeg instead.').format(format)
      return imshow(a, format='jpeg')
    else:
      raise
  return disp

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

        
def clamp(x, minimum, maximum):
    return max(minimum, min(x, maximum))
    
def drawLatent(image,latents,x,y,x2,y2, color=(255,0,0,100)):
  buffer = PIL.Image.new('RGBA', image.size, (0,0,0,0))
   
  draw = ImageDraw.Draw(buffer)
  cy = (y+y2)/2
  draw.rectangle([x,y,x2,y2],fill=(255,255,255,180), outline=(0,0,0,180))
  for i in range(len(latents)):
    mx = x + (x2-x)*(float(i)/len(latents))
    h = (y2-y)*latents[i]*0.1
    h = clamp(h,cy-y2,y2-cy)
    draw.line((mx,cy,mx,cy+h),fill=color)
  return PIL.Image.alpha_composite(image,buffer)
             
  
def createImageGrid(images, scale=0.25, rows=1):
   w,h = images[0].size
   w = int(w*scale)
   h = int(h*scale)
   height = rows*h
   cols = ceil(len(images) / rows)
   width = cols*w
   canvas = PIL.Image.new('RGBA', (width,height), 'white')
   for i,img in enumerate(images):
     img = img.resize((w,h), PIL.Image.ANTIALIAS)
     canvas.paste(img, (w*(i % cols), h*(i // cols))) 
   return canvas

def convertZtoW(latent, truncation_psi=0.7, truncation_cutoff=9):
  dlatent = Gs.components.mapping.run(latent, None) # [seed, layer, component]
  dlatent_avg = Gs.get_var('dlatent_avg') # [component]
  for i in range(truncation_cutoff):
    dlatent[0][i] = (dlatent[0][i]-dlatent_avg)*truncation_psi + dlatent_avg
    
  return dlatent

def interpolate(zs, steps):
   out = []
   for i in range(len(zs)-1):
    for index in range(steps):
     fraction = index/float(steps) 
     out.append(zs[i+1]*fraction + zs[i]*(1-fraction))
   return out

# Taken from https://github.com/alexanderkuk/log-progress
def log_progress(sequence, every=1, size=None, name='Items'):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = int(size / 200)     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{name}: {index} / ?'.format(
                        name=name,
                        index=index
                    )
                else:
                    progress.value = index
                    label.value = u'{name}: {index} / {size}'.format(
                        name=name,
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = "{name}: {index}".format(
            name=name,
            index=str(index or '?')
        )



In [4]:
#@title Additional functions for point cloud generation



def rotate(plane, angle):
    theta = np.radians(angle)
    # rot_x = np.array([[1, 0, 0], [0, np.cos(theta), -np.sin(theta)], [0, np.sin(theta), np.cos(theta)]])
    rot_y = np.array([[np.cos(theta), 0, np.sin(theta)], [0, 1, 0], [-np.sin(theta), 0, np.cos(theta)]])
    # rot_z = np.array([[np.cos(theta), -np.sin(theta), 0], [np.sin(theta), np.cos(theta), 0], [0, 0, 1]])
    return plane.dot(rot_y)


def img2points(img, d=0, a=0, thr=255):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    th = cv2.adaptiveThreshold(gray, thr, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 4)
    w, h = gray.shape
    depth = 1 - th / 255
    new_depth = depth + d
    pixel_x, pixel_y = np.meshgrid(np.linspace(0, w - 1, w),
                                   np.linspace(0, h - 1, h))
    camera_points = np.array([pixel_x, pixel_y, new_depth]).transpose(1, 2, 0).reshape(-1, 3)
    color_points = img.reshape(-1, 3)
    if a % 360 != 0:
        camera_points = rotate(camera_points, a)

    valid_depth_ind = np.where(depth.flatten() > 0)[0]
    camera_points = camera_points[valid_depth_ind, :]
    color_points = color_points[valid_depth_ind, :]
    color_points = color_points.astype(int)

    return camera_points, color_points


def write_pointcloud(filename, xyz_points, rgb_points=None):
    """ creates a .ply file of the generated point clouds 
    """

    assert xyz_points.shape[1] == 3, 'Input XYZ points should be Nx3 float array'
    if rgb_points is None:
        rgb_points = np.ones(xyz_points.shape).astype(np.uint8) * 255
    assert xyz_points.shape == rgb_points.shape, 'Input RGB colors should be Nx3 float array and have same size as input XYZ points'

    # Write header of .ply file
    with open(filename, 'wb') as fid:
        fid.write(bytes('ply\n', 'utf-8'))
        fid.write(bytes('format binary_little_endian 1.0\n', 'utf-8'))
        fid.write(bytes(f'element vertex {xyz_points.shape[0]}\n', 'utf-8'))
        fid.write(bytes('property float x\n', 'utf-8'))
        fid.write(bytes('property float y\n', 'utf-8'))
        fid.write(bytes('property float z\n', 'utf-8'))
        fid.write(bytes('property uchar red\n', 'utf-8'))
        fid.write(bytes('property uchar green\n', 'utf-8'))
        fid.write(bytes('property uchar blue\n', 'utf-8'))
        fid.write(bytes('end_header\n', 'utf-8'))

        # Write 3D points to .ply file
        for i in range(xyz_points.shape[0]):
            fid.write(bytearray(struct.pack("fffBBB", xyz_points[i, 0], xyz_points[i, 1], xyz_points[i, 2],
                                            rgb_points[i, 0], rgb_points[i, 1],
                                            rgb_points[i, 2])))


def sparse_image(img, level):
    new_image = np.zeros(img.shape, dtype=np.uint8)
    new_image.fill(255)
    new_image[::level,::level] = img[::level,::level]
    return new_image

In [ ]:
#@title Mount Google Drive
#@markdown Mount Google Drive to load pretrained models and to save the results.

#@markdown After running this cell you will get the link. Follow the link, grant access to your Drive and copy auth code.

#@markdown Paste the code to the input below and press Enter

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Load StyleGAN2 model
#@markdown Provide the path to pretrained StyleGAN2 model (.pkl file)
network_pkl = "/content/drive/MyDrive/stylegan/models/ucl-000032.pkl" #@param {type: "string"}
dnnlib.tflib.init_tf()

with dnnlib.util.open_url(network_pkl) as fp:
    _G, _D, Gs = pickle.load(fp)
noise_vars = [var for name, var in Gs.components.synthesis.vars.items() if name.startswith('noise')]

In [ ]:
#@title Image generation

#@markdown Number of images
num_images = 10 #@param {type: "integer"}
#@markdown Number of interpolation steps
num_steps = 20 #@param {type: "integer"}
#@markdown Directory to save the results. This folder will be used to save images, video and point cloud
results_dir = "/content/drive/MyDrive/workshops/ucl/pc4" #@param {type: "string"}


if results_dir=="":
  raise Exception("Please specify results directory!")
else:
  if results_dir[-1] != "/":
    results_dir += "/"
  results_images = f'{results_dir}images/'
  results_video = f'{results_dir}video/'
  results_cloud = f'{results_dir}cloud/'
  if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    os.makedirs(results_images)
    os.makedirs(results_video)
    os.makedirs(results_cloud)
  else:
    raise Exception("This folder already exists!")

seeds = list(np.random.randint(4294967295, size=num_images))
seeds = seeds + [seeds[0]]
zs = generate_zs_from_seeds(seeds)

imgs = generate_images(interpolate(zs, num_steps), 1)

saveImgs(imgs, results_images)
with imageio.get_writer(results_video+'interpolate.mp4', mode='I') as writer:
    for image in log_progress(list(imgs), name = "Creating animation"):
        writer.append_data(np.array(image))
print('Done!')

In [ ]:
#@title Point Cloud Generation

#@markdown Take every n-th point
skip = 10 #@param {type: "integer"}
#@markdown Distance between layers (if >1 distance will be bigger, if from 0 to 1 it will be scaled down)
distance = 1 #@param {type: "number"}
#@markdown Set point cloud file name
cloud_name = "maps-test-point-cloud" #@param {type: "string"}


if cloud_name=="":
  raise Exception("Please specify point cloud file name!")

total = num_images*num_steps
angle_shift = 360 / total
angle_current = -angle_shift
all_points = np.empty((0, 3))
all_colors = np.empty((0, 3))

for i in range(total):
    if i % skip != 0:
        continue
    # print(f'{results_images}{i}.png')
    image = cv2.imread(f'{results_images}{i}.png', 1)
    image = sparse_image(image, skip)
    points, colors = img2points(image, d=i*distance, a=0)
    all_points = np.concatenate((all_points, points))
    all_colors = np.concatenate((all_colors, colors))


write_pointcloud(f'{results_cloud}{cloud_name}.ply', all_points, all_colors.astype(np.uint8))
print(f'Cloud saved to {results_cloud}{cloud_name}.ply')

In [ ]:
#@title Images to Point Cloud

#@markdown If you don't want to generate new imeges with GAN, but want to create point cloud from already existing images you can run this cell. Just be sure the first four cells are already executed.

#@markdown Take every n-th point
skip = 10 #@param {type: "integer"}
#@markdown Distance between layers (if >1 distance will be bigger, if from 0 to 1 it will be scaled down)
distance = 1 #@param {type: "number"}
#@markdown Image directory
image_dir = "/content/drive/MyDrive/workshops/ucl/CLIP" #@param {type: "string"}
#@markdown Directory to save point cloud
results_cloud = "/content/drive/MyDrive/workshops/ucl/pc4/cloud" #@param {type: "string"}
#@markdown Set point cloud file name
cloud_name = "maps-test-point-cloud-d3" #@param {type: "string"}


if image_dir=="":
  raise Exception("Please specify image directory")
if image_dir[-1] != "/":
    image_dir += "/"
if results_cloud=="":
  results_cloud= '/content/'
if results_cloud[-1] != "/":
    results_cloud += "/"
if not os.path.exists(results_cloud):
  os.makedirs(results_cloud)
if cloud_name=="":
  raise Exception("Please specify point cloud file name!")

img_files = sorted(glob.glob(f'{image_dir}*.png')+glob.glob(f'{image_dir}*.jpg'))
total = len(img_files)
angle_shift = 360 / total
angle_current = -angle_shift
all_points = np.empty((0, 3))
all_colors = np.empty((0, 3))



for i, fname in enumerate(img_files):
    if i % skip != 0:
        continue
    image = cv2.imread(fname, 1)
    image = cv2.resize(image, (1024,1024), interpolation = cv2.INTER_LINEAR)
    image = sparse_image(image, skip)
    points, colors = img2points(image, d=i*distance, a=0)
    all_points = np.concatenate((all_points, points))
    all_colors = np.concatenate((all_colors, colors))


write_pointcloud(f'{results_cloud}{cloud_name}.ply', all_points, all_colors.astype(np.uint8))
print(f'Cloud saved to {results_cloud}{cloud_name}.ply')